# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 28 2022 9:20AM,240601,10,SO94300,"Senseonics, Incorporated",Released
1,Jun 28 2022 9:16AM,240599,20,ALU0007164,Alumier Labs Inc.,Released
2,Jun 28 2022 9:16AM,240599,20,ALU0007166,Alumier Labs Inc.,Released
3,Jun 28 2022 9:16AM,240599,20,ALUR527052956,Alumier Labs Inc.,Released
4,Jun 28 2022 9:16AM,240599,20,ALUR762483969,Alumier Labs Inc.,Released
5,Jun 28 2022 9:16AM,240599,20,ALUR851567943,Alumier Labs Inc.,Released
6,Jun 28 2022 9:16AM,240599,20,ALUR568303500,Alumier Labs Inc.,Released
7,Jun 28 2022 9:16AM,240599,20,ALUR518393973,Alumier Labs Inc.,Released
8,Jun 28 2022 9:16AM,240599,20,ALU0007167,Alumier Labs Inc.,Released
9,Jun 28 2022 9:16AM,240599,20,ALU0007168,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,240597,Released,5
42,240598,Released,16
43,240599,Released,21
44,240600,Released,3
45,240601,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240597,NaN,NaN,5.0
240598,NaN,NaN,16.0
240599,NaN,NaN,21.0
240600,NaN,NaN,3.0
240601,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240476,0.0,1.0,0.0
240492,0.0,1.0,0.0
240497,0.0,0.0,1.0
240502,0.0,1.0,0.0
240524,18.0,20.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240476,0,1,0
240492,0,1,0
240497,0,0,1
240502,0,1,0
240524,18,20,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240476,0,1,0
1,240492,0,1,0
2,240497,0,0,1
3,240502,0,1,0
4,240524,18,20,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240476,,1,
1,240492,,1,
2,240497,,,1
3,240502,,1,
4,240524,18,20,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated"
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.
22,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance
25,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation
41,Jun 28 2022 8:39AM,240597,10,Emerginnova
46,Jun 28 2022 8:32AM,240595,10,"Methapharm, Inc."
50,Jun 28 2022 8:30AM,240577,10,Ivaoes Animal Health
51,Jun 28 2022 8:19AM,240594,16,Sartorius Bioprocess Solutions
58,Jun 28 2022 8:06AM,240593,16,Sartorius Bioprocess Solutions
62,Jun 28 2022 8:01AM,240592,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",,,1
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,,,21
2,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance,,,3
3,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation,,,16
4,Jun 28 2022 8:39AM,240597,10,Emerginnova,,,5
5,Jun 28 2022 8:32AM,240595,10,"Methapharm, Inc.",,,4
6,Jun 28 2022 8:30AM,240577,10,Ivaoes Animal Health,,,1
7,Jun 28 2022 8:19AM,240594,16,Sartorius Bioprocess Solutions,,,7
8,Jun 28 2022 8:06AM,240593,16,Sartorius Bioprocess Solutions,,,4
9,Jun 28 2022 8:01AM,240592,19,"GUSA Granules USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",1,,
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,21,,
2,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance,3,,
3,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation,16,,
4,Jun 28 2022 8:39AM,240597,10,Emerginnova,5,,
5,Jun 28 2022 8:32AM,240595,10,"Methapharm, Inc.",4,,
6,Jun 28 2022 8:30AM,240577,10,Ivaoes Animal Health,1,,
7,Jun 28 2022 8:19AM,240594,16,Sartorius Bioprocess Solutions,7,,
8,Jun 28 2022 8:06AM,240593,16,Sartorius Bioprocess Solutions,4,,
9,Jun 28 2022 8:01AM,240592,19,"GUSA Granules USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",1,,
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,21,,
2,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance,3,,
3,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation,16,,
4,Jun 28 2022 8:39AM,240597,10,Emerginnova,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,21.0,NaN,NaN
2,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance,3.0,NaN,NaN
3,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation,16.0,NaN,NaN
4,Jun 28 2022 8:39AM,240597,10,Emerginnova,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 28 2022 9:20AM,240601,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jun 28 2022 9:16AM,240599,20,Alumier Labs Inc.,21.0,0.0,0.0
2,Jun 28 2022 9:14AM,240600,10,Beach Patient Assistance,3.0,0.0,0.0
3,Jun 28 2022 9:09AM,240598,10,ISDIN Corporation,16.0,0.0,0.0
4,Jun 28 2022 8:39AM,240597,10,Emerginnova,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3608689,219.0,4.0,0.0
16,721663,11.0,1.0,0.0
17,240550,1.0,0.0,0.0
19,1924555,53.0,8.0,23.0
20,1443312,49.0,47.0,18.0
21,1443216,4.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3608689,219.0,4.0,0.0
1,16,721663,11.0,1.0,0.0
2,17,240550,1.0,0.0,0.0
3,19,1924555,53.0,8.0,23.0
4,20,1443312,49.0,47.0,18.0
5,21,1443216,4.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,219.0,4.0,0.0
1,16,11.0,1.0,0.0
2,17,1.0,0.0,0.0
3,19,53.0,8.0,23.0
4,20,49.0,47.0,18.0
5,21,4.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,219.0
1,16,Released,11.0
2,17,Released,1.0
3,19,Released,53.0
4,20,Released,49.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,23.0,18.0,0.0
Executing,4.0,1.0,0.0,8.0,47.0,2.0
Released,219.0,11.0,1.0,53.0,49.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21
0,Completed,0.0,0.0,0.0,23.0,18.0,0.0
1,Executing,4.0,1.0,0.0,8.0,47.0,2.0
2,Released,219.0,11.0,1.0,53.0,49.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21
0,Completed,0.0,0.0,0.0,23.0,18.0,0.0
1,Executing,4.0,1.0,0.0,8.0,47.0,2.0
2,Released,219.0,11.0,1.0,53.0,49.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()